#Tosi et al Benchmark

This notebook generates models from the <a name="ref-1"/>[(Tosi et al., 2015)](#cite-tosi2015community) Benchmark in Underworld2. The Underworld results are compared to the benchmark model run on Fenics. Input files for the Fenics models were provided by Petra Maierova. The comparison of Underworld2 and Fenics is useful because both are FEM codes which can be built Python environment and both interface to PetSc to solve the linear systems. 

In [2]:
#%%javascript
#IPython.load_extensions('calico-document-tools');

In [3]:
# =============================================================================
# CONSTANTS and PARAMETERS
# =============================================================================
RA  = 1e2        # Rayleigh number
TS  = 0          # surface temperature
TB  = 1          # bottom boundary temperature (melting point)
CASE = 2 # select identifier of the testing case (1-5)
YSTRESS = 1.0    # yield stress=1 in cases 1-4, =3.0..5.0 in case 5
ETA_T = 1e5
ETA_Y = 10
ETA0 = 1e-3
TMAX = 3.0
IMAX = 1000
XDIV, YDIV = 40, 40 # number of horizontal and vertical divisions
C_CFL = 0.5      # Courant number
#dt = Constant(1e-10)# initial value of time step

In [4]:
# RT PIC - classic and nearest neighbour
import underworld as uw
import math
from underworld import function as fn
import glucifer.pylab as plt
import numpy as np
import os

In [5]:
dim = 2

# create mesh objects
elementMesh = uw.mesh.FeMesh_Cartesian( elementType=("linear","constant"), 
                                         elementRes=(XDIV,YDIV), 
                                           minCoord=(0.,0.), 
                                           maxCoord=(1.,1.)  )
linearMesh   = elementMesh
constantMesh = elementMesh.subMesh 

In [6]:
# create fevariables
velocityField    = uw.fevariable.FeVariable( feMesh=linearMesh,   nodeDofCount=dim )
pressureField    = uw.fevariable.FeVariable( feMesh=constantMesh, nodeDofCount=1 )
temperatureField = uw.fevariable.FeVariable( feMesh=linearMesh,   nodeDofCount=1 )

In [7]:
# create some dummy fevariables for doing top and bottom boundary integrations
topField    = uw.fevariable.FeVariable( feMesh=linearMesh,   nodeDofCount=1)
bottomField    = uw.fevariable.FeVariable( feMesh=linearMesh,   nodeDofCount=1)

topField.data[:] = 0.
bottomField.data[:] = 0.

# Set top / bottom boundaries to 1
for index in linearMesh.specialSets["MinJ_VertexSet"]:
    bottomField.data[index] = 1.
for index in linearMesh.specialSets["MaxJ_VertexSet"]:
    topField.data[index] = 1.

#ICs and BCs

In [8]:
# Initialise data.. Note that we are also setting boundary conditions here
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.
temperatureField.data[:] = 0.

# Setup temperature initial condition via numpy arrays
A = 0.01
import math
#Note that width = height = 1
tempNump = temperatureField.data
for index, coord in enumerate(linearMesh.data):
    pertCoeff = (1- coord[1]) + A*math.cos( math.pi * coord[0] ) * math.sin( math.pi * coord[1] )
    tempNump[index] = pertCoeff;
    

In [34]:
figtemp = plt.Figure()
figtemp.Surface(temperatureField, elementMesh)
figtemp.show()

OSError: [Errno 7] Argument list too long

In [10]:
# Get list of special sets.
# These are sets of vertices on the mesh. In this case we want to set them as boundary conditions.
linearMesh.specialSets.keys()

['AllWalls',
 'MaxI_VertexSet',
 'MinJ_VertexSet',
 'MinI_VertexSet',
 'MaxJ_VertexSet']

In [11]:
# Get the actual sets 
#
#  HJJJJJJH
#  I      I
#  I      I
#  I      I
#  HJJJJJJH
#  
#  Note that H = I & J 

# Note that we use operator overloading to combine sets
IWalls = linearMesh.specialSets["MinI_VertexSet"] + linearMesh.specialSets["MaxI_VertexSet"]
JWalls = linearMesh.specialSets["MinJ_VertexSet"] + linearMesh.specialSets["MaxJ_VertexSet"]

In [12]:
# Now setup the dirichlet boundary condition
# Note that through this object, we are flagging to the system 
# that these nodes are to be considered as boundary conditions. 
# Also note that we provide a tuple of sets.. One for the Vx, one for Vy.
freeslipBC = uw.conditions.DirichletCondition(     variable=velocityField, 
                                              nodeIndexSets=(IWalls,JWalls) )

# also set dirichlet for temp field
tempBC = uw.conditions.DirichletCondition(     variable=temperatureField, 
                                              nodeIndexSets=(JWalls,) )

In [13]:
# Set temp boundaries 
# on the boundaries
for index in linearMesh.specialSets["MinJ_VertexSet"]:
    temperatureField.data[index] = TB
for index in linearMesh.specialSets["MaxJ_VertexSet"]:
    temperatureField.data[index] = TS

#Viscosity




In [14]:
#Make some necessary arrays for the the 

secinvCopy = fn.tensor.second_invariant( 
                    fn.tensor.symmetric( 
                        velocityField.gradientFn ))

coordinate = fn.input()

In [15]:
#Remember to use floats everywhere when setting up functions

#Linear viscosities
viscosityl1 = fn.math.exp(math.log(ETA_T)*-1*temperatureField)
viscosityl2 = fn.math.exp((math.log(ETA_T)*-1*temperatureField) + (1.-coordinate[1]*math.log(ETA_Y)))

viscosityFn1 = viscosityl1 #This one always gets passed to the first velcotity solve

#Von Mises effective viscosity
viscosityp = ETA0 + YSTRESS/(secinvCopy/math.sqrt(0.5)) #extra factor to account for underworld second invariant form


if CASE == 1:
    viscosityFn2 = viscosityFn1
elif CASE == 2:
    viscosityFn2 = 2./(1./viscosityl1 + 1./viscosityp)
elif CASE == 3:
    viscosityFn2 = viscosityl2
else:
    viscosityFn2 = 2./(1./viscosityl2 + 1./viscosityp)


In [35]:
# lets take a look at the eta function
figEta = plt.Figure()
figEta.Surface(viscosityl2, linearMesh)
figEta.show()

OSError: [Errno 7] Argument list too long

In [17]:
densityFn = RA*temperatureField
# Define our gravity using a python tuple (this will be automatically converted to a function)
gravity = ( 0.0, 1.0 )
# now create a buoyancy force vector.. the gravity tuple is converted to a function 
# here via operator overloading
buoyancyFn = gravity*densityFn

##Build the Stokes system, solvers, advection-diffusion

In [18]:
#We first set up a linear Stokes system to get the initial velocity
stokesPIC = uw.systems.Stokes(velocityField=velocityField, 
                              pressureField=pressureField,
                              conditions=[freeslipBC,],
                              viscosityFn=fn.exception.SafeMaths(viscosityFn1), 
                              bodyForceFn=buoyancyFn)

In [19]:
#Solve for initial velocity field - so we can begin the non-linear part
stokesPIC.solve()

In [20]:
# Setup the Stokes system again, now with linear or nonlinear visocity viscosity.
stokesPIC2 = uw.systems.Stokes(velocityField=velocityField, 
                              pressureField=pressureField,
                              conditions=[freeslipBC,],
                              viscosityFn=fn.exception.SafeMaths(viscosityFn2), 
                              bodyForceFn=buoyancyFn )

In [21]:
solver = uw.systems.Solver(stokesPIC2)

In [22]:
# Create advdiff system
advDiff = uw.systems.AdvectionDiffusion( temperatureField, velocityField, diffusivity=1., conditions=[tempBC,] )
# Also create some integral objects which are used to calculate statistics.
v2sum_integral  = uw.utils.Integral( feMesh=linearMesh, fn=fn.math.dot(velocityField, velocityField) ) 
volume_integral = uw.utils.Integral( feMesh=linearMesh, fn=1. )

##Metrics for benchmark

For cases 1-4, participants were asked to report a number of diagnostic quantities to be measured after reaching steady state:

* Average temp... $$  \langle T \rangle  = \int^1_0 \int^1_0 T \, dxdy $$
* Top and bottom Nusselt numbers... $$N = \int^1_0 \frac{\partial T}{\partial y} \rvert_{y=0/1} \, dx$$
* RMS velocity over the whole domain, surface and max velocity at surface
* max and min viscosity over the whole domain
* average rate of work done against gravity...$$\langle W \rangle = \int^1_0 \int^1_0 T u_y \, dx dy$$
* and the average rate of viscous dissipation...$$\langle \Phi \rangle = \int^1_0 \int^1_0 \tau_{ij} \dot \epsilon_{ij} \, dx dy$$

* In steady state, if thermal energy is accurately conserved, the difference between $\langle W \rangle$ and $\langle \Phi \rangle / Ra$ must vanish, so also reported is the percentage error: 

$$ \delta = \frac{\lvert \langle W \rangle - \frac{\langle \Phi \rangle}{Ra} \rvert}{max \left(  \langle W \rangle,  \frac{\langle \Phi \rangle}{Ra}\right)} \times 100% $$

In [23]:
def avg_temp():
    temp = uw.utils.Integral(temperatureField, linearMesh)
    return temp.integrate()[0]

def nuss():
    nt = []
    nb = []
    grad = temperatureField.gradientFn 
    gradarray = grad.evaluate(linearMesh)
    for index in linearMesh.specialSets["MaxJ_VertexSet"]:
        nt.append(gradarray[index][1]*(1./XDIV))
    for index in linearMesh.specialSets["MinJ_VertexSet"]:
        nb.append(gradarray[index][1]*(1./XDIV))
    n1 = sum(nt)
    n0 = sum(nb)
    return abs(n1), abs(n0)
    
def rms():
    squared = uw.utils.Integral(fn.math.dot(velocityField,velocityField), linearMesh)
    area = uw.utils.Integral(1.,linearMesh)
    return math.sqrt(squared.integrate()[0]/area.integrate()[0])

def rms_surf():
    xvelocityField = fn.math.dot(velocityField,np.array([1.,0.]))
    squared = uw.utils.Integral(fn.math.dot(topField, fn.math.dot(xvelocityField,xvelocityField)), linearMesh)
    toparea = uw.utils.Integral((topField*1.),linearMesh)
    return math.sqrt(squared.integrate()[0]/toparea.integrate()[0])

def max_vx_surf():
    surf_vels =[]
    for index in linearMesh.specialSets["MaxJ_VertexSet"]:
        surf_vels.append(velocityField.data[index][0])
    return max(surf_vels)

def gravwork():
    dw = uw.utils.Integral(temperatureField*velocityField[1], linearMesh)
    return dw.integrate()[0]

#note that viscosityFn2 is used here, so this is not case independent. 
def viscdis():
    secinv = fn.tensor.second_invariant( 
                    fn.tensor.symmetric( 
                        velocityField.gradientFn ))
    sinner = fn.math.dot(secinv,secinv)
    vd = uw.utils.Integral((4.*viscosityFn2*sinner), linearMesh)
    return vd.integrate()[0]

def visc_extr():
    testfn = fn.view.min_max(viscosityFn2)
    #evaluate on the mesh
    testfn.evaluate(linearMesh) 
    vmax, vmin = testfn.max_global(), testfn.min_global()
    return vmax, vmin
    
        

#linearMesh.specialSets["MinJ_VertexSet"]()

In [24]:
testfn = fn.view.min_max(viscosityFn2)
#evaluate on the mesh
testfn.evaluate(linearMesh) 
testfn.max_global()

1.9290136272831258

In [25]:
test = fn.math.dot(velocityField,velocityField)
type(test)

underworld.function.math.dot

In [26]:
surf_vels =[]
for index in linearMesh.specialSets["MaxJ_VertexSet"]:
    surf_vels.append(velocityField.data[index][0])
max(surf_vels)

0.12625138816353604

In [27]:
nuss()
max_vx_surf()
#grad = temperatureField.gradientFn 


0.12625138816353604

##solve using internal Picard iteration

In [28]:
# Stepping. Initialise time and timestep.
import time
realtime = 0.
step = 0
timevals = []
vrmsvals = []

In [29]:
start = time.clock()
fname = "uw_results_case" + str(CASE) + ".dat"
f_o = open(os.path.join("./",fname), 'w')
# Perform steps
while step<400:
    #Enter non-linear loop
    solver.solve(nonLinearIterate=True)
    dt = advDiff.get_max_dt()
    if step == 0:
        dt = 0.
    # Advect using this timestep size   
    advDiff.integrate(dt)
    # Increment
    realtime += dt
    step += 1
    timevals.append(time)
    # Calculate the Metrics
    Avg_temp = avg_temp()
    Rms = rms()
    Rms_surf = rms_surf()
    Max_vx_surf = max_vx_surf()
    Gravwork = gravwork()
    Viscdis = viscdis()
    nu1, nu0 = nuss()
    etamax, etamin = visc_extr()
    f_o.write((11*'%-15s ' + '\n') % (realtime, Viscdis, nu0, nu1, Avg_temp, Rms,Rms_surf,Max_vx_surf,Gravwork, etamax, etamin))
    if step % 20 == 0:
        print 'step =',step, 'Rms =', Rms, 'Nu bottom', nu0, 'Nu top', nu1
f_o.close()

step = 20 Rms = 12.4993748888 Nu bottom 1.05048159744 Nu top 1.02503942472
step = 40 Rms = 26.8989061589 Nu bottom 1.13297100916 Nu top 1.02506751333
step = 60 Rms = 44.2437669678 Nu bottom 1.26888347339 Nu top 1.02508473522
step = 80 Rms = 64.1808653355 Nu bottom 1.45158734387 Nu top 1.02510862097
step = 100 Rms = 82.9017840385 Nu bottom 1.6652195648 Nu top 1.0251223944
step = 120 Rms = 96.0385649298 Nu bottom 1.88843245931 Nu top 1.02514128851
step = 140 Rms = 102.286493991 Nu bottom 2.09729365855 Nu top 1.02517041133
step = 160 Rms = 103.695540592 Nu bottom 2.27822292972 Nu top 1.02521507108
step = 180 Rms = 103.407009902 Nu bottom 2.44198672647 Nu top 1.02528560474
step = 200 Rms = 102.89589449 Nu bottom 2.59771726542 Nu top 1.02539948267
step = 220 Rms = 103.036492618 Nu bottom 2.75013035549 Nu top 1.02561443617
step = 240 Rms = 104.197424208 Nu bottom 2.89874308774 Nu top 1.02605930739
step = 260 Rms = 107.332499642 Nu bottom 3.04113649843 Nu top 1.02699371432
step = 280 Rms = 11

#Figures

In [30]:
#switch off to run in Parallel

In [32]:
# lets check FEM solution
velMag = plt.Figure()
velMag.Surface(fn.math.dot(velocityField,velocityField), linearMesh)
velMag.show()

OSError: [Errno 7] Argument list too long

Figure 1. Steady state snapshots of temperature, viscosity, RMS velocity, second invariant of the stress tensor, and (right column) corresponding laterally averaged profiles for Case 1 obtained with the code Underworld. 

Figure 2.. As in Figure 1, but for Case 2... The four profiles can be found in the Data Set S2 of the supporting information.

#References

<a name="cite-tosi2015community"/><sup>[^](#ref-1) </sup>Tosi, Nicola and Stein, Claudia and Noack, Lena and H&uuml;ttig, Christian and Maierov&aacute;, Petra and Samuel, Henri and Davies, DR and Wilson, CR and Kramer, SC and Thieulot, Cedric and others. 2015. _A community benchmark for viscoplastic thermal convection in a 2-D square box_.



<!--bibtex

@article{tosi2015community,
  title={A community benchmark for viscoplastic thermal convection in a 2-D square box},
  author={Tosi, Nicola and Stein, Claudia and Noack, Lena and H{\"u}ttig, Christian and Maierov{\'a}, Petra and Samuel, Henri and Davies, DR and Wilson, CR and Kramer, SC and Thieulot, Cedric and others},
  journal={Geochemistry, Geophysics, Geosystems},
  year={2015},
  publisher={Wiley Online Library}
}

... Other Bibtex entries go here.

-->